In [1]:
import re
import os
import requests
from urllib import request
import threading
from queue import Queue


os.chdir(r"C:\Users\acer\Desktop\images")
# 生产者类：获取网页的地址及图片的链接、名字
class Producer(threading.Thread):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"
    }
    def __init__(self, page_queue, img_queue, *args, **kwargs):
        super(Producer, self).__init__(*args, **kwargs)
        self.page_queue = page_queue
        self.img_queue = img_queue

    def parse_age(self, url):
        rsp = requests.get(url, headers=self.headers)
        html = rsp.content.decode("utf-8")
        image_addr = re.findall(r'<img\sreferrerpolicy=.+?data-original="(.*?)".*?>', html, re.DOTALL)
        image_name = re.findall(r'<img\sreferrerpolicy=.+?alt="(.*?)".*?>', html, re.DOTALL)
        image_name = [re.sub(r'[\?？\.。,！!\*]', '', i) for i in image_name]
        suffixs = [(os.path.splitext(i))[1] for i in image_addr]
        for addr, name, suffix in zip(image_addr, image_name, suffixs):
            file_name = name + suffix
            self.img_queue.put((addr, file_name))

    def run(self):
        while True:
            if self.page_queue.empty():
                break
            url = self.page_queue.get()
            self.parse_age(url)


# 消费者类：从图片地址下载图片并保存到save_dir文件下
class Consumer(threading.Thread):

    def __init__(self, page_queue, img_queue, *args, **kwargs):
        super(Consumer, self).__init__(*args, **kwargs)
        self.page_queue = page_queue
        self.img_queue = img_queue

    def run(self):
        while True:
            if self.img_queue.empty() and self.page_queue.empty():
                break
            img_addr, img_name = self.img_queue.get()
            request.urlretrieve(img_addr, img_name)
            print(img_name,"下载完成")

def main():
    # 设置存放页面和图片地址的队列
    page_queue = Queue(100)
    img_queue =Queue(1000)
    for i in range(1, 3):
        url =  "https://www.doutula.com/photo/list/?page={}".format(i)
        page_queue.put(url)
    for i in range(5):
        p = Producer(page_queue, img_queue)
        p.start()

    for i in range(5):
        c = Consumer(page_queue, img_queue)
        c.start()



if __name__ == "__main__":
    main()
    print("what's the matter")

what's the matter
